In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By

In [14]:
#웹트라이버 자동다운로드
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [15]:
browser=set_chrome_driver()



[Korean Sentence Splitter]: 

====== WebDriver manager ======
[Korean Sentence Splitter]: ====== WebDriver manager ======
Current google-chrome version is 96.0.4664
[Korean Sentence Splitter]: Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
[Korean Sentence Splitter]: Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\82103\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
[Korean Sentence Splitter]: Driver [C:\Users\82103\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache


In [203]:
browser.get("https://www.youtube.com/watch?v=fTd3mFViKTU") # 찾을 영상 링크
#https://www.youtube.com/watch?v=3YGQtszkn1Q
#https://www.youtube.com/watch?v=fe8PrSKZu-I
#https://www.youtube.com/watch?v=gUTuWXQgnXk
#https://www.youtube.com/watch?v=fTd3mFViKTU
#https://www.youtube.com/watch?v=yemXn4uJzMk

# 코로나 대학생활

In [106]:
def infinite_loop():
    #무한 스크롤 내리기
    last_page_height = browser.execute_script("return document.documentElement.scrollHeight")

    while True:
        browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.0)
        new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        if new_page_height == last_page_height:
            time.sleep(1.0)
            if new_page_height == browser.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

In [204]:
infinite_loop()

In [205]:
comments = browser.find_elements(By.CSS_SELECTOR, "#content-text")

In [206]:
len(comments)

13

In [187]:
# 처음 df 생성
text_df2=pd.DataFrame(index=range(len(comments)), columns=['text'])
for i in range(len(comments)):
    text_df2.loc[i,'text']=comments[i].text

In [207]:
# 기존 df에서 데이터 합치기
text=[]
for i in range(len(comments)):
    text.append(comments[i].text)
for lst in text:
    text_df2=text_df2.append({'text':lst}, ignore_index=True)

In [208]:
text_df2

,text
0,캠퍼스 낭만은 이제 없네 어쩌네 하지만\n사회생활하다보면 대학때가 그래도 낭만적인 ...
1,02:19 얼마나 슬프면 앞도 안보일까 ㅜㅜ
2,2020년에 초에 군대 입대한 2000년생들 부럽다..새내기때 할 거 다하고 군대에...
3,내 나이 마흔 넷 .... 그래도 돌아보면 봄냄새 나면서 풋풋하고 뭘 해도 뭔...
4,사실상 현고3보단 ..작년에 대학교 입학하신분들이 젤 불쌍한거임
...,...
176,3수에 코로나학번 ㅠㅠ 나도 코로나학번이지만 저형 너무 안타깝다 ㅠㅠ
177,01년생. 세월호 안타까운 사고로 중고등학교 수학여행X. 마지막 문이과(재수ㅈ). ...
178,ㅋㄱㄱㄱㄱㄱㄱㄱㅋㄱㄱㄱㄱㄱㄱㅋㄱㄱㄱㄱㄱㄱㅋㄱㄱㄱ나이따
179,인생이그래 IMF 학번도있다\n코로나학번? 더살아바 이보다 더한거 징그럽게 많은게 ...


In [152]:
text_df2.to_csv("대학생인간관계text.csv", encoding='utf-8-sig', index=False)

In [153]:
text_df=pd.read_csv("대학생인간관계text.csv", encoding='utf-8-sig')

In [2]:
import dataload

In [3]:
from konlpy.tag import Okt
from kss import split_sentences
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from pykospacing import Spacing
from hanspell import spell_checker
from textrankr import TextRank
import re
from collections import Counter
from typing import List

In [4]:
# 네이버 블로그 크롤링
dataload.My_naver_api_ID = 'ZMfTEU85Ct0GirFZsOJQ'  
dataload.My_naver_api_Secret = 'oxmHIWyLsn'  
data=dataload.api_naver_TL('대학생활 인간관계','kin')

In [5]:
data

,Title,Link
0,<b>대학생활 인간관계</b>,https://kin.naver.com/qna/detail.naver?d1id=11...
1,"대학교(동아리,시간표,<b>대학생활</b>,<b>인간관계</b>)여러가지...",https://kin.naver.com/qna/detail.naver?d1id=11...
2,<b>대학생활 인간관계</b>에 대해서,https://kin.naver.com/qna/detail.naver?d1id=8&...
3,<b>대학생활</b>...<b>인간관계</b> 고민입니다.,https://kin.naver.com/qna/detail.naver?d1id=20...
4,<b>대학생활 인간관계</b>가 너무힘듭니다.,https://kin.naver.com/qna/detail.naver?d1id=11...
...,...,...
950,고등학교친구와 <b>대학</b>동기,https://kin.naver.com/qna/detail.naver?d1id=11...
951,<b>대학생활</b>에 대한 회의를 느껴요.. 수능을 다시 볼까요.....,https://kin.naver.com/qna/detail.naver?d1id=20...
952,"고민,자퇴 <b>대학생활</b>",https://kin.naver.com/qna/detail.naver?d1id=11...
953,<b>대학</b>교친구...<b>인간관계</b>..,https://kin.naver.com/qna/detail.naver?d1id=11...


In [11]:
data.loc[0,'Link']

'https://kin.naver.com/qna/detail.naver?d1id=11&dirId=110502&docId=254650909&qb=64yA7ZWZ7IOd7ZmcIOyduOqwhOq0gOqzhA==&enc=utf8&section=kin.qna&rank=32&search_sort=0&spq=0'

In [6]:
kin_data=dataload.make_body(data, 'kin')

In [23]:
kin_data

,Q_title,Q_content,A_content_1,A_content_2,A_content_3,A_content_4,A_content_5
0,코딩학습/IT개발,,제가 생각하는 대학의 인간관계는.. 바람 같네요.학기중에는 하하호호 친하게 지내다가...,NaN,NaN,NaN,NaN
1,고등학교학습,"1. 대학교동아리는 가입하는게 좋을까요?(뭐 듣어보면 가입하면 선배가 생겨서좋고, ...",1. 대학가서 공부만 하는사람이 어딧어요!이것저것 많이 경험해봐야죠!지금이 아니면 ...,"1. 동아리 가입하면 장,단점이 다있죠. 님이 말씀하신대로 공부,성적에 지장을 좀 ...",NaN,NaN,NaN
2,대학생활 인간관계에 대해서,안녕하세요 저는 12학번 지난 학기에 복학한 남자인데요 1학년때는 잘몰랐는데 군대다...,음. 어떻게 보면 현명하고 어떻게 보면 정말 뻔한 속물적 성향들에 혼란을 느끼시는거...,안녕하세요 저도 경험자입니다사실대로 말하면 요즘애들은 다 자기밖에 모릅니다어른들이 ...,NaN,NaN,NaN
3,,,NaN,NaN,NaN,NaN,NaN
4,코딩학습/IT개발,이번에 갓 13학번 신입생이된 남자입니다. 저는 중고등학교때부터 게임만하다와서 1...,얼마나 힘드실지 마음이 아픕니다.. 한번 인간관계에 실패를 경험했다해서 절대 모든게...,네이버 편입 전문 카페입니다. 음....어떻게 보면 많은 고등학생들이 대학생활에 ...,NaN,NaN,NaN
...,...,...,...,...,...,...,...
950,코딩학습/IT개발,고등학교때만해도반애들하고 잘어울리고 재밌게 놀고 공부도 열심히 했었는데 대학생이 되...,네..어떤 심정인지 잘 알겠습니다.사실 저같은 경우도 조금 시골에서 고등학교를 다니...,NaN,NaN,NaN,NaN
951,,,NaN,NaN,NaN,NaN,NaN
952,코딩학습/IT개발,안녕하세요 현재 대학 1학년을 재학중인 학생입니다 다름이 아니라 매일 심각하게(?)...,자퇴 글 올라오니까...하이에나들처럼 자퇴하라는 장삿꾼들 답글 엄청 달리네요.하고 ...,원하지 않는 전공이고 대학이라면차라리 편입준비를 해보시는게 어떠세요??편입조건은 온...,안녕하세요. 교육부정식인가 원격평생교육원 학습플래너 조수명팀장 입니다. 그런 문...,안녕하세요~~~학점집도의 양집도 입니다.~~~~원치 않는 전공에서 스트레스 받으시...,안녕하세요 교육부정식인가 평생교육원 소속 박영지 입니다^^질문하신 사항 답해드리...
953,고등학교학습,제가 얼마전에 오티를 갔다왓습니다.근데 왜이렇게 말을하기싫은지..말을붙이기가 참 힘...,싸우세요,오늘만 열심히 사세요. 내일일은 내일 생각하세요.오늘은 열심히 공부하고 학교가서 사...,NaN,NaN,NaN


In [29]:
kin_data=kin_data.dropna(subset=['A_content_1']).reset_index(drop=True)

In [33]:
kin_data.drop(['A_content_2','A_content_3','A_content_4','A_content_5','Q_title'],axis=1, inplace=True)

In [39]:
kin_data.head()

,Q_content,A_content_1,text_re
0,,제가 생각하는 대학의 인간관계는.. 바람 같네요.학기중에는 하하호호 친하게 지내다가...,
1,"1. 대학교동아리는 가입하는게 좋을까요?(뭐 듣어보면 가입하면 선배가 생겨서좋고, ...",1. 대학가서 공부만 하는사람이 어딧어요!이것저것 많이 경험해봐야죠!지금이 아니면 ...,1.대학교 동아리는 가입하는 게 좋을까요? 뭐 듣어보면 가입하면 선배가 생겨서 좋고...
2,안녕하세요 저는 12학번 지난 학기에 복학한 남자인데요 1학년때는 잘몰랐는데 군대다...,음. 어떻게 보면 현명하고 어떻게 보면 정말 뻔한 속물적 성향들에 혼란을 느끼시는거...,안녕하세 요저는 12학번 지난 학기에 복학한 남자인데요 1학년 때는 잘 몰랐는데 군...
3,이번에 갓 13학번 신입생이된 남자입니다. 저는 중고등학교때부터 게임만하다와서 1...,얼마나 힘드실지 마음이 아픕니다.. 한번 인간관계에 실패를 경험했다해서 절대 모든게...,이번에 갓 13학번 신입생이 된 남자입니다. 저는 중고등학교 때부터 게임만 하다와서...
4,저는 21살 대학생 남잔데요 제가 성격이 내성적이고 소심해서 지금까지 친구를 사귀...,제 생각에는 여자친구보다 우선 친구들을 사귀는게 더 중요한것 같네요.친구들을 자주 ...,저는 21살 대학생 남잔데요제가 성격이 내성적이고 소심해서 지금까지 친구를 사귀지 ...


In [40]:
# 의미없는 데이터 정제
kin_data['text_re']=np.nan
for i in range(len(kin_data)):
    try:
        norm = re.compile('[가-힣|0-9|.|?|a-z|A-Z]+').findall(kin_data.loc[i,'Q_content'])
        st = ''.join(norm)
        space=Spacing(rules=['코노'])
        st=space(st)
        kin_data.loc[i,'text_re']=st
    except:
        print(i)

In [41]:
data=kin_data.copy()

In [50]:
data['text_re_sen']=''
for i in range(len(data)):
    temp=[]
    for st in split_sentences(data.loc[i,'text_re']):
        temp.append(st)
    for st in temp:
        data.loc[i,'text_re_sen']+=(str(st)+'.')

In [44]:
class MyTokenizer:
    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = text.split()
        return tokens
class OktTokenizer:
    okt: Okt = Okt()

    def __call__(self, text: str) -> List[str]:
        tokens: List[str] = self.okt.phrases(text)
        return tokens
mytokenizer: MyTokenizer = MyTokenizer()
textrank: TextRank = TextRank(mytokenizer)

In [52]:
data.loc[2,'text_re_sen']

'안녕하세 요저는 12학번 지난 학기에 복학한 남자인데요 1학년 때는 잘 몰랐는데 군대 다녀오고 하니까 대학교인 간관계라는 게 참가식적이고 타산적으로 보이더라고요 그냥 서로 경쟁하고 못 믿고 말하나에 주의해야 하는 그런 관계랄까요?.친하다고 할 수 있는 무리가 둘 있는데 또 그 무리끼리 서로 안친해서 서로 헐뜯고 이 간질하고 마음속에 있는 얘기들 하기엔 신뢰도 안 가고 그리고 그 친구들 전부 제학창 시절 친구들과 비교할 때 진실되지 않고 자기하고 싶은 말 돌려서 은근히 욕한다 던지제가 누구랑 좀 친해지려고 하면 쟤는 못 믿을 놈이고 뒷통수친다 약 점 잡아서.. 이런 식으로 말하는 얘가 있는데 결과적으로는 그 얘가 제가 한 말 남에게 해서 관계 안 좋게 만들고 또 그 얘가 욕했던 무리들과도 편견이 생겨버려서인지 친하게 지내기 싫네요.그냥 차라리 공부해서 더 좋은 대학으로 가고 싶은 마음입니다..일단 질문은 진짜 친구 운운하면서 남들 조심하라는 놈이 진짜 친구이고 저를 정말 생각해서 다른 무리들이 내게 뒤에서 한 말을 전해주는 걸까요?.그리고 거리감 느껴 진다고 말한 걸 다른 친구에게 말해서 사이 단절시켰는데 스터디 회장으로서 어쩔 수 없었다는 게 진심일까요?.그리고 그 다른 무리놈들이 제게 제 친구들에 대해 욕하거나 가식적으로 행동하는 모습을 보여주는 데 이렇게 모두가 거짓된 관계로 보일 때는 어떻게 해야 할까요?.정말 심란하네요 쓰다 보니.'

In [53]:
# 예시
k: int = 10
summarized: str = textrank.summarize(data.loc[3,'text_re_sen'], k)
print(summarized)

그러다 보니 점점 인간관계가 재밌어 지구 친구들도 너무 너무 좋아지는 겁니다
하지만 너무 안일했습니다
친구들에게 너무 많은 정을 줘버린 거죠
제가 너무 많은 걸 털어놓았나 봅니다
이런 소리가 점점 귀에 들리더군요그러다 저랑 사귀었던 첫사랑
너무 안 좋은 일이 갑자기 겹치니 너무 힘듭니다
이런 일이 있고 나니
많이 울기도 했구요
이런 소리 하지 말아주세요
이런 제 감정 어떻게 다스려야 할까요


In [54]:
data['textrank_10']=''
k: int = 10
for i in range(len(data)):

    summarized: str = textrank.summarize(data.loc[i,'text_re_sen'], k)
        
    data.loc[i,'textrank_10']=summarized

In [55]:
data.head()

,Q_content,A_content_1,text_re,text_re_sen,textrank_10
0,,제가 생각하는 대학의 인간관계는.. 바람 같네요.학기중에는 하하호호 친하게 지내다가...,,,
1,"1. 대학교동아리는 가입하는게 좋을까요?(뭐 듣어보면 가입하면 선배가 생겨서좋고, ...",1. 대학가서 공부만 하는사람이 어딧어요!이것저것 많이 경험해봐야죠!지금이 아니면 ...,1.대학교 동아리는 가입하는 게 좋을까요? 뭐 듣어보면 가입하면 선배가 생겨서 좋고...,1.대학교 동아리는 가입하는 게 좋을까요?.뭐 듣어보면 가입하면 선배가 생겨서 좋고...,1.대학교 동아리는 가입하는 게 좋을까요\n공부에 지장도 되고 2.대학교 시간표 학...
2,안녕하세요 저는 12학번 지난 학기에 복학한 남자인데요 1학년때는 잘몰랐는데 군대다...,음. 어떻게 보면 현명하고 어떻게 보면 정말 뻔한 속물적 성향들에 혼란을 느끼시는거...,안녕하세 요저는 12학번 지난 학기에 복학한 남자인데요 1학년 때는 잘 몰랐는데 군...,안녕하세 요저는 12학번 지난 학기에 복학한 남자인데요 1학년 때는 잘 몰랐는데 군...,안녕하세 요저는 12학번 지난 학기에 복학한 남자인데요 1학년 때는 잘 몰랐는데 군...
3,이번에 갓 13학번 신입생이된 남자입니다. 저는 중고등학교때부터 게임만하다와서 1...,얼마나 힘드실지 마음이 아픕니다.. 한번 인간관계에 실패를 경험했다해서 절대 모든게...,이번에 갓 13학번 신입생이 된 남자입니다. 저는 중고등학교 때부터 게임만 하다와서...,이번에 갓 13학번 신입생이 된 남자입니다..저는 중고등학교 때부터 게임만 하다와서...,그러다 보니 점점 인간관계가 재밌어 지구 친구들도 너무 너무 좋아지는 겁니다\n하지...
4,저는 21살 대학생 남잔데요 제가 성격이 내성적이고 소심해서 지금까지 친구를 사귀...,제 생각에는 여자친구보다 우선 친구들을 사귀는게 더 중요한것 같네요.친구들을 자주 ...,저는 21살 대학생 남잔데요제가 성격이 내성적이고 소심해서 지금까지 친구를 사귀지 ...,저는 21살 대학생 남잔데요제가 성격이 내성적이고 소심해서 지금까지 친구를 사귀지 ...,저는 21살 대학생 남잔데요제가 성격이 내성적이고 소심해서 지금까지 친구를 사귀지 ...


In [57]:
data.loc[5,'textrank_10']

'군대 있을 땐 복학하면 대학교 1학년 때보다 더 재밌게 생활해보자 하면서 복학했는데 아는 친구도 2명밖에 없고 그마저도 저는 다른 반에 가면서 아는 사람이 한 명도 없네요\n저 번 일주일 동안 하루에 세 마디씩 하고 온 것 같습니다\n역시나 저랑 맞는 사람 아니면 사귀기 참 힘들어요\n1학년 때는 내가 성격이 좀 바꼈나보다\n이렇게 생각했는데 그냥 나랑 맞던 사람이 많아서 그런 것 같네요\n억지로 사귀려했다가 피곤해서 그만둔 경우도 있었습니다\n슬슬 우울해지려고 하고 앞으로도이 러면 회사가 선어떡하지 걱정도 되고 그러네요'

In [58]:
# 명사 추출
okt=Okt()
all_text=[]
for i in range(len(data)):
    if len(str(data.loc[i,'textrank_10']))>1:
        sub_lst=okt.pos(str(data.loc[i,'textrank_10']))
        temp=[]
        for n,key in sub_lst:
            if (key=='Noun') and len(n)>1 :
                temp.append(n)
        all_text=all_text+temp

conut=Counter(all_text)

In [67]:
na_list=conut.most_common(100)

In [66]:
na_list

[('친구', 1014),
 ('대학', 606),
 ('사람', 429),
 ('생각', 390),
 ('정말', 349),
 ('생활', 340),
 ('인간관계', 251),
 ('그냥', 220),
 ('지금', 217),
 ('고등학교', 215),
 ('학교', 201),
 ('공부', 181),
 ('다른', 180),
 ('대학교', 178),
 ('학년', 171),
 ('혼자', 165),
 ('성격', 158),
 ('관계', 130),
 ('진짜', 120),
 ('고민', 117),
 ('정도', 115),
 ('문제', 113),
 ('때문', 105),
 ('선배', 103),
 ('연락', 101),
 ('이제', 97),
 ('걱정', 90),
 ('학기', 87),
 ('요즘', 87),
 ('마음', 82),
 ('동아리', 81),
 ('졸업', 79),
 ('시간', 76),
 ('자퇴', 74),
 ('질문', 73),
 ('수업', 72),
 ('학생', 71),
 ('얘기', 69),
 ('하나', 69),
 ('현재', 69),
 ('거의', 68),
 ('조언', 62),
 ('성적', 61),
 ('스트레스', 61),
 ('보고', 61),
 ('별로', 60),
 ('이번', 60),
 ('아이', 60),
 ('항상', 59),
 ('대해', 58),
 ('입학', 57),
 ('먼저', 57),
 ('취업', 57),
 ('건가', 56),
 ('중학교', 55),
 ('계속', 55),
 ('군대', 54),
 ('여자', 54),
 ('상황', 54),
 ('처음', 52),
 ('방법', 52),
 ('대학생', 51),
 ('남자', 51),
 ('자신', 50),
 ('가지', 49),
 ('무슨', 49),
 ('준비', 49),
 ('하나요', 48),
 ('나이', 48),
 ('다시', 47),
 ('가면', 47),
 ('원래', 47),
 ('동기', 47),
 ('사실', 46),
 

In [68]:
# 명사 & 형용사 추출
okt=Okt()
all_text=[]
for i in range(len(data)):
    if len(str(data.loc[i,'textrank_10']))>1:
        sub_lst=okt.pos(str(data.loc[i,'textrank_10']))
        temp=[]
        for n,key in sub_lst:
            if (key=='Noun' or key=='Adjective') and len(n)>1 :
                temp.append(n)
        all_text=all_text+temp

conut=Counter(all_text)

In [71]:
na_list=conut.most_common(500)

In [72]:
na_list

[('친구', 1014),
 ('대학', 606),
 ('사람', 429),
 ('생각', 390),
 ('정말', 349),
 ('생활', 340),
 ('인간관계', 251),
 ('이런', 235),
 ('어떻게', 235),
 ('입니다', 232),
 ('그냥', 220),
 ('지금', 217),
 ('고등학교', 215),
 ('있는', 214),
 ('같은', 206),
 ('학교', 201),
 ('그런', 195),
 ('공부', 181),
 ('다른', 180),
 ('대학교', 178),
 ('학년', 171),
 ('혼자', 165),
 ('성격', 158),
 ('관계', 130),
 ('진짜', 120),
 ('고민', 117),
 ('정도', 115),
 ('문제', 113),
 ('때문', 105),
 ('선배', 103),
 ('같습니다', 102),
 ('연락', 101),
 ('이제', 97),
 ('같아요', 97),
 ('없고', 91),
 ('걱정', 90),
 ('없는', 88),
 ('학기', 87),
 ('좋은', 87),
 ('요즘', 87),
 ('마음', 82),
 ('친한', 82),
 ('동아리', 81),
 ('있습니다', 81),
 ('졸업', 79),
 ('시간', 76),
 ('자퇴', 74),
 ('질문', 73),
 ('있을까요', 73),
 ('수업', 72),
 ('있는데', 71),
 ('학생', 71),
 ('얘기', 69),
 ('하나', 69),
 ('현재', 69),
 ('거의', 68),
 ('같고', 67),
 ('솔직히', 62),
 ('조언', 62),
 ('성적', 61),
 ('스트레스', 61),
 ('보고', 61),
 ('있고', 60),
 ('별로', 60),
 ('이번', 60),
 ('아이', 60),
 ('항상', 59),
 ('대해', 58),
 ('많은', 58),
 ('입학', 57),
 ('먼저', 57),
 ('취업', 57),
 ('어떤', 56),

In [ ]:
# 아래는 유튜브 정제

In [154]:
# 의미없는 데이터 정제
text_df['text_re']=np.nan
for i in range(len(text_df)):
    try:
        norm = re.compile('[가-힣|0-9|.|?|a-z|A-Z]+').findall(text_df.loc[i,'text'])
        st = ''.join(norm)
        st = spell_checker.check(st)
        space=Spacing(rules=['코노'])
        st2=space(st[2])
        text_df.loc[i,'text_re']=st2
    except:
        print(i)

843
1166
1182
1435
1482
1495
1744
2022
2050


In [155]:
text_df.tail(20)

,text,text_re
2214,삼각근이 매력적이시네요,삼각근이 매력적이 시네요
2215,여자나 남자나 능력,여자나 남자나 능력
2216,어떤 직무이냐에 따라 다른듯,어떤 직무이냐에 따라 다른 듯
2217,오프숄더 너무예뻐용,오프숄더 너무 예뻐요
2218,이런거 듣지말고 직접 느껴라,이런 거 듣지 말고 직접 느껴라
2219,20대입니다,20대입니다
2220,근데...인트로에 참고만 해달라는거 무슨 뜻 이에요?,근데... 인트로에 참고만 해달라는 거 무슨 뜻이에요?
2221,그냥 다 아닥하고 니 공부할거 해라. \n고3에게 좋은 조언이 되었습니다.,그냥 다 아닥하고 너 공부할 거 해라. 고3에게 좋은 조언이 되었습니다.
2222,능력있어도 안붙는사람은 안붙음 관심도 없던데 그사람은,능력 있어도 안 붙는 사람은 안 붙음 관심도 없던데 그 사람은
2223,맞아요언냐.....,맞아요 언니.....


In [178]:
# 명사 추출
okt=Okt()
all_text=[]
for i in range(len(text_df)):
    if len(str(text_df.loc[i,'text_re']))>1:
        sub_lst=okt.pos(str(text_df.loc[i,'text_re']))
        temp=[]
        for n,key in sub_lst:
            if (key=='Noun' or key=='Adjective') and len(n)>1 :
                temp.append(n)
        if '영상' in temp:
            continue
        all_text=all_text+temp

conut=Counter(all_text)

In [179]:
na_list=conut.most_common(100)

In [180]:
na_list

[('친구', 1527),
 ('사람', 535),
 ('생각', 409),
 ('공감', 291),
 ('언니', 227),
 ('진짜', 217),
 ('좋은', 210),
 ('정말', 197),
 ('같아요', 188),
 ('관계', 156),
 ('있는', 156),
 ('그런', 150),
 ('그냥', 139),
 ('마음', 131),
 ('지금', 121),
 ('대학', 119),
 ('얘기', 117),
 ('같은', 112),
 ('연락', 110),
 ('위로', 110),
 ('같아', 104),
 ('혼자', 103),
 ('시간', 102),
 ('인생', 98),
 ('이야기', 94),
 ('이런', 92),
 ('없는', 90),
 ('이십', 90),
 ('자기', 87),
 ('인간관계', 83),
 ('자신', 81),
 ('서로', 78),
 ('상처', 77),
 ('나이', 75),
 ('입니다', 74),
 ('정은', 71),
 ('고민', 69),
 ('인맥', 68),
 ('인연', 67),
 ('걱정', 66),
 ('다른', 65),
 ('조언', 65),
 ('많은', 62),
 ('하나', 60),
 ('능력', 59),
 ('기분', 58),
 ('말씀', 58),
 ('항상', 57),
 ('느낌', 54),
 ('때문', 53),
 ('대해', 53),
 ('고등학교', 51),
 ('계속', 51),
 ('보고', 50),
 ('본인', 50),
 ('요즘', 50),
 ('사랑', 49),
 ('질투', 49),
 ('노력', 48),
 ('있는데', 48),
 ('결혼', 48),
 ('있어요', 47),
 ('순간', 45),
 ('주변', 45),
 ('절교', 45),
 ('경험', 44),
 ('상황', 44),
 ('문제', 44),
 ('이해', 44),
 ('시즌', 44),
 ('부분', 43),
 ('가족', 43),
 ('아닌', 43),
 ('생활', 42),
 ('필요

In [100]:
# 친구 명사 있는 문장만 따오기
friends=[]
okt=Okt()
for i in range(len(text_df)):
    nouns=okt.nouns(text_df.loc[i,'text_re'])
    if '대학' in nouns:
        lst=[]#['대학','친구','같은','가면','고등학교','진짜']
        st=text_df.loc[i,'text_re']
        for name in lst:
            st=st.replace(name,"")
        friends.append(st)

In [101]:
friends[10:70]

['대학 들어와서 오티 때 친해진 애들과 계속 톡 방 만들고 우정 유지 중.. 근데 진짜 솔메이트 1명 빼고는 뭐랄까 더 좀 서먹하긴 해요',
 '이름 있는 대학가면 진짜 친구 만들기 쉬움 일단 첫 만남부터 서로가 어느 정도 수준의 사람이라는 걸 알 수 있다 즉 탐색 비용이 들지 않아 그 이후 시간 지나친 해지면 대학 친구는 겉친구 다 이런 생각 절대 안 한다 대학 친구가 중요한 인맥이란 걸 알기 때문에 그리고 대학 친구는 겉친이고 고등학교 친구는 찐 친이다는 대학 진학률 낮은 시절 이야기지 대학 친구랑도 경험하는 게 많고 취업준비도 같이 하고 비슷한 라인의 회사 취직할 텐데',
 '개인적으로 고등학교 친구들은 말 안 해도 통하는 친구 대학 친구들은 다양한 관점의 친구 고등학교 친구들이랑은 더 공감하면서 이야기할 수 있다면 대학 친구들과는 더 배우면서 세상이 넓어지는 기분이었습니다 둘 다 좋아요 어느 때 만난 친구들이든 만날 친구는 만나게 되더라고요',
 '잘 맞는 고등학교 중학교 때 친구들이 있다는 건 정말 행운입니다. 그때 친구들이 진짜 친구고 서로 거리낌 없이 털어놓을 수 있는 찐 친이거든요 그러나 현실은 가는 길도 다르고 가는 대학도 다르고 이래서 이때 친구들이랑 멀어지고 소수만 남습니다. 대학 가서 사귄 친구들은 중고딩 때 친구들이랑은 좀 느낌이 다릅니다.사회생활 같이 하는 동료 성격이 있어요',
 '근데 확률로 따지면 고등학생 친구가 대학 친구보다 돈독할 확률은 높음.. 대학 친구는 일단 기본적으로는 술 친구에서 시작하는 거라 고등학교 친구보다 노력이 더 필요합니다. 아무래도 같이 있는 시간이 상대적으로 적 다 보네 게다가 틀어질 갈등 요소 또한 많고',
 '현 고 딘은 대학생 때 친구가 없어도 되니까 우선 대학이 나가고 싶네요.....',
 '대학 가면 12학년 때는 다들 두루두루 친하게 잘 지 내다가 3학년쯤 되면 알아서 다들 서로 잘 맞는 사람들끼리 모여 다님 꼭 동기들이랑 만 친하게 지 낼 필요는 없음 선배든 동기든 후배든 잘 맞으면 끝',


In [86]:
# Q_data tf-idf 사용후 keyword 추출
vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(text_df['text_re'])

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

Q_tfidf = []
for i, sent in enumerate(text_df['text_re']):
    tfidf = sorted([ (token, sp_matrix[i, word2id[token]])for token in sent.split()],
           key = lambda x: x[1] , reverse = True)
    Q_tfidf.append(tfidf)

# Q_tfidf 중복 제거 후 가중치 순으로 정렬
Q_id = [sorted(list(set(lst)), key = lambda x:x[1], reverse =True) for lst in Q_tfidf]

In [89]:
Q_id

[[('잊지', 0.32273037441456287),
  ('추천도', 0.32273037441456287),
  ('달아줘라', 0.32273037441456287),
  ('많다', 0.32273037441456287),
  ('고생', 0.3062044092608752),
  ('것들', 0.2944790503838532),
  ('댓글', 0.2779530852301656),
  ('말고', 0.24970176119945595),
  ('다들', 0.24643937691196063),
  ('코로나', 0.21882356061372324),
  ('주제', 0.21101909316878673),
  ('싶은', 0.18478924350074924),
  ('때문에', 0.18478924350074924),
  ('하고', 0.15386430247351135),
  ('끝나면', 0.13482935794206755),
  ('에', 0.0)],
 [('쓰지', 0.3971177879828391),
  ('소원이에요', 0.3971177879828391),
  ('내는', 0.3767826870862506),
  ('것만으로도', 0.3511634897937599),
  ('않고', 0.3420196024927616),
  ('편하게', 0.3342885559368918),
  ('나가서', 0.2960653043036824),
  ('밖에', 0.2791903704468143),
  ('마스크', 0.158068847660839),
  ('지', 0.0)],
 [('쓰라고', 0.27114342641658906),
  ('화내면서', 0.27114342641658906),
  ('하시는데', 0.27114342641658906),
  ('가니까', 0.27114342641658906),
  ('선생님들은', 0.25725905986218456),
  ('정작', 0.25725905986218456),
  ('있으니까', 0.2572590598621845